Importações necessárias e instalações necessárias

In [8]:
!pip install pydantic
!pip install pydantic[email]

In [5]:
from enum import auto, IntFlag
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)

Criação da classe Role, que define os papéis de usuário
Author, Editor e Developer recebem valores automáticos, enquanto Admin é a combinação das 3

In [6]:
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

Criação da classe User, que define um modelo de usuário com:
name, uma string simples, utilizando Arjan como exemplo
email, um campo validado como email
password, um campo que será ocultado em logs, visto que é algo sensível
e role (o padrão é none). Cada um utiliza Field() para adicionar metadados como exemplos

In [7]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(default=None, description="The role of the user")

Criação da função validate, que recebe um dicionário de dados e tenta validá-los, se der certo, imprime o objeto do usuário e se falhar itera sobre cada erro específico

In [9]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid")
        for error in e.errors():
            print(error)

Agora a criação da função main, que define um exemplo de dados válidos com nome, email e senha e também exemplos de dados inválidos com email e senhas "errados" e chama a função validate com ambos conjuntos

In [10]:
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    validate(good_data)
    validate(bad_data)

In [11]:
if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}
